In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

Matplotlib is building the font cache; this may take a moment.


In [4]:
path = 'Analytics/Customer Projects - ScoringData3/pnc_retro/toLNRS/QB11414/FilesFromCustomer/cc_apps_2018_2021_LN_Upload.zip'

In [5]:
from dsgtools import azure
data = pd.read_adls(path, compression = "zip", reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data.shape)

(2000000, 857)


## Overall Cleaning

In [7]:
data["address"] = np.where(data.address1.notnull(), np.where(data.address2.notnull(), data.address1 + " " + data.address2, data.address1), data.address1)
Input_FirstName = "first_name"
Input_LastName = "last_name"
Input_StreetAddress = "address"
Input_City = "city"
Input_State = "state"
Input_Zip = "zip_code"
Input_SSN = "ssn"

data["sufficient_input"] = np.where((data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_Zip].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_City].notnull() & data[Input_State].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_SSN].notnull()), 1, 0)
freq(data["sufficient_input"])

,Count,Pct,Cuml Count,Cuml Pct
sufficient_input,,,,
0,146.0,0.000073,146.0,0.000073
1,1999854.0,0.999927,2000000.0,1.000000


In [8]:
data = data[data["sufficient_input"] == 1]
data["application_dt_fmt"] = data.application_dt.str.slice(0,4) + data.application_dt.str.slice(5,7)
data["application_dt_fmt_dt"] = data.application_dt.str.slice(0,4) + data.application_dt.str.slice(5,7) + data.application_dt.str.slice(8,10)

In [9]:
data["dpd_90P_CO"] = np.where(data.DPD_90_WiN_12.isnull(), -1, 
                              np.where((data.DPD_90_WiN_12 =='1') | (data.DPD_120_WiN_12 =='1') | (data.CO_BNK_WiN_12=='1'), 1, 0))
print(data.shape)
freq(data.dpd_90P_CO)

(1999854, 862)


,Count,Pct,Cuml Count,Cuml Pct
dpd_90P_CO,,,,
-1,819171.0,0.409615,819171.0,0.409615
0,1160666.0,0.580375,1979837.0,0.989991
1,20017.0,0.010009,1999854.0,1.000000


In [10]:
data = data.sort_values(by = ['last_name', 'address', 'city', 'state', 'zip_code', 'pri_phone', 'ssn', 'birth_date', 'application_dt_fmt_dt', "dpd_90P_CO"], ascending = False)
print(data.shape)
data = data.drop_duplicates(subset = ['last_name', 'address', 'city', 'state', 'zip_code', 'pri_phone', 'ssn', 'birth_date', 'application_dt_fmt_dt'], keep = "first")
print(data.shape)

(1999854, 862)
(1971367, 862)


In [12]:
time_list = ['2021-10', '2021-11', '2021-12']
data = data[~data.application_dt.str.slice(0, 7).isin(time_list)]
print(data.shape)

(1857583, 862)


In [26]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/pnc_11414_full_file.parquet",
             overwrite = True)

In [14]:
from dsgtools import azure
data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/pnc_11414_full_file.parquet")
print(data.shape)

(1857583, 862)


In [16]:
path = "Analytics/Customer Projects - ScoringData3/pnc_retro/toLNRS/QB11414/FilesFromCustomer/Rachel_work/cc_apps_2018_2021_LN_Upload_total_dobnew3.csv.gz"
dob = pd.read_adls(path, compression = "gzip", reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(dob.shape)

(2000000, 16)


In [21]:
print(dob.application_id.is_unique)
dob = dob[["application_id", "birth_date_y"]]
dob.columns = ["application_id", "birth_date_updated"]
data = data.merge(dob, on = "application_id")
print(data.shape)

True
(1857583, 863)


In [22]:
from datetime import datetime
print(data.birth_date.isnull().sum())
data["birth_date_fmt"] = pd.to_datetime(data.birth_date_updated, format = "%m-%d-%Y", errors = "coerce")
print(data.birth_date_fmt.isnull().sum())

0
5690


In [24]:
data["birth_date_fmt_dt"] = data['birth_date_fmt'].astype(str).str.slice(0,4) + data['birth_date_fmt'].astype(str).str.slice(5,7) + data['birth_date_fmt'].astype(str).str.slice(8,10)

## For validation

In [27]:
freq(data.appl_status_grp)

,Count,Pct,Cuml Count,Cuml Pct
appl_status_grp,,,,
Approved,2.0,0.000001,2.0,0.000001
Booked,1131293.0,0.609013,1131295.0,0.609015
Declined,718797.0,0.386953,1850092.0,0.995967
Withdrawn,7491.0,0.004033,1857583.0,1.000000


In [28]:
data = data[data.appl_status_grp == "Booked"]
print(data.shape)
freq(data.closed_WiN_12, data.lost_stolen_WiN_12, cross = False, observed = True)

(1131293, 865)


Count       Pct  Cuml Count  Cuml Pct
closed_WiN_12 lost_stolen_WiN_12                                           
1             0                     38815.0  0.034310     38815.0  0.034310
              1                       222.0  0.000196     39037.0  0.034507
0             0                   1078627.0  0.953446   1117664.0  0.987953
              1                      7057.0  0.006238   1124721.0  0.994191
Missing       Missing                6572.0  0.005809   1131293.0  1.000000

In [29]:
data = data[(data.closed_WiN_12 == "0") & (data.lost_stolen_WiN_12 == "0")]
print(data.shape)
freq(data.fraud_deceased_WiN_12, data.DPD_90_WiN_12, data.DPD_120_WiN_12, data.CO_BNK_WiN_12, 
     cross = False, observed = True)

(1078627, 865)


Count  \
fraud_deceased_WiN_12 DPD_90_WiN_12 DPD_120_WiN_12 CO_BNK_WiN_12              
0                     0             0              0              1060412.0   
                                                   1                 3691.0   
                      1             0              0                 2457.0   
                                                   1                  259.0   
                                    1              0                 3736.0   
                                                   1                 7286.0   
1                     0             0              0                   26.0   
                                                   1                  416.0   
                      1             0              0                   11.0   
                                                   1                  117.0   
                                    1              0                   10.0   
                                                   1                  206.0   

                                                                       Pct  \
fraud_deceased_WiN_12 DPD_90_WiN_12 DPD_120_WiN_12 CO_BNK_WiN_12             
0                     0             0              0              0.983113   
                                                   1              0.003422   
                      1             0              0              0.002278   
                                                   1              0.000240   
                                    1              0              0.003464   
                                                   1              0.006755   
1                     0             0              0              0.000024   
                                                   1              0.000386   
                      1             0              0              0.000010   
                                                   1              0.000108   
                                    1              0              0.000009   
                                                   1              0.000191   

                                                                  Cuml Count  \
fraud_deceased_WiN_12 DPD_90_WiN_12 DPD_120_WiN_12 CO_BNK_WiN_12               
0                     0             0              0               1060412.0   
                                                   1               1064103.0   
                      1             0              0               1066560.0   
                                                   1               1066819.0   
                                    1              0               1070555.0   
                                                   1               1077841.0   
1                     0             0              0               1077867.0   
                                                   1               1078283.0   
                      1             0              0               1078294.0   
                                                   1               1078411.0   
                                    1              0               1078421.0   
                                                   1               1078627.0   

                                                                  Cuml Pct  
fraud_deceased_WiN_12 DPD_90_WiN_12 DPD_120_WiN_12 CO_BNK_WiN_12            
0                     0             0              0              0.983113  
                                                   1              0.986535  
                      1             0              0              0.988813  
                                                   1              0.989053  
                                    1              0              0.992516  
                                                   1              0.999271  
1                     0             0              0              0.999295  
                                                   1    

In [30]:
data = data[data.fraud_deceased_WiN_12 =="0"]
time_list = ['2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', 
             '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
             '2021-06', '2021-07', '2021-08', '2021-09', ]
data = data[data.application_dt.str.slice(0, 7).isin(time_list)]
print(data.shape)

(507908, 865)


In [31]:
freq(data.USE)

,Count,Pct,Cuml Count,Cuml Pct
USE,,,,
PRI,486228.0,0.957315,486228.0,0.957315
SEC,21680.0,0.042685,507908.0,1.000000


In [32]:
data = data[data.USE == "PRI"]
print(data.shape)

(486228, 865)


In [33]:
data[['last_name', 'address1', "address2", 'city', 'state', 'zip_code', 'pri_phone', 'ssn', 'birth_date']].isnull().sum()

last_name          0
address1           0
address2      461602
city               0
state              0
zip_code           0
pri_phone          0
ssn                0
birth_date         0
dtype: int64

In [34]:
from dsgtools import azure
# data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/pnc_cleaned_11_17.parquet")
bivariate(data.application_dt.str.slice(0, 7), data.dpd_90P_CO)

tag            dpd_90P_CO                                                
stats                   N      PctN     Sum      Mean       WoE        IV
application_dt                                                           
2019-08           35515.0  0.073042   740.0  0.020836  0.256331  0.005445
2019-09           36990.0  0.076075   720.0  0.019465  0.186840  0.002910
2019-10           41138.0  0.084606   836.0  0.020322  0.230807  0.005048
2019-11           39059.0  0.080331   739.0  0.018920  0.157906  0.002164
2019-12           34780.0  0.071530   698.0  0.020069  0.218030  0.003785
2020-01           36604.0  0.075282   641.0  0.017512  0.079119  0.000490
2020-02           33026.0  0.067923   593.0  0.017956  0.104598  0.000782
2020-03           21564.0  0.044350   378.0  0.017529  0.080134  0.000296
2020-04            6425.0  0.013214    75.0  0.011673 -0.332387  0.001248
2020-05            4190.0  0.008617    55.0  0.013126 -0.213574  0.000355
2020-06            4757.0  0.009783    60.0  0.012613 -0.254000  0.000559
2020-07            7710.0  0.015857   117.0  0.015175 -0.066473  0.000068
2020-08            8869.0  0.018240    95.0  0.010711 -0.419336  0.002634
2020-09            8764.0  0.018024   118.0  0.013464 -0.187832  0.000581
2020-10           10302.0  0.021188   118.0  0.011454 -0.351553  0.002218
2020-11            9590.0  0.019723    86.0  0.008968 -0.598786  0.005360
2020-12            9508.0  0.019555    98.0  0.010307 -0.458226  0.003314
2021-01           12224.0  0.025140   129.0  0.010553 -0.434400  0.003870
2021-02           11988.0  0.024655   152.0  0.012679 -0.248685  0.001355
2021-03           17461.0  0.035911   286.0  0.016379  0.011117  0.000004
2021-04           15240.0  0.031343   297.0  0.019488  0.188069  0.001216
2021-05           15299.0  0.031465   225.0  0.014707 -0.098291  0.000290
2021-06           16409.0  0.033748   213.0  0.012981 -0.224892  0.001534
2021-07           16863.0  0.034681   145.0  0.008599 -0.641172  0.010606
2021-08           16194.0  0.033305   155.0  0.009571 -0.533018  0.007384
2021-09           15759.0  0.032411   108.0  0.006853 -0.869824  0.016534
Missing               0.0  0.000000     0.0  0.000000  0.000000  0.000000
Total            486228.0  1.000000  7877.0  0.016200  0.000000  0.080051

In [35]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/pnc_cleaned_11_17.parquet", overwrite = True)

### Add Weight

In [36]:
data["weight"] = 1
data_1 = data[(data.dpd_90P_CO == 1)].copy()
data_0 = data[(data.dpd_90P_CO == 0)].copy()
print(data_1.shape)
print(data_0.shape)

(7877, 866)
(478351, 866)


In [37]:
freq(data.dpd_90P_CO)

,Count,Pct,Cuml Count,Cuml Pct
dpd_90P_CO,,,,
0,478351.0,0.9838,478351.0,0.9838
1,7877.0,0.0162,486228.0,1.0000


In [38]:
data_0_sample = data_0.sample(n = 250_000 - data_1.shape[0], random_state=1)
data_0_sample['weight'] = data_0.shape[0]/(250_000 - data_1.shape[0])
print(data_0_sample['weight'].unique())
print(data_0_sample.shape)

[1.97565287]
(242123, 866)


In [39]:
data = pd.concat([data_1, data_0_sample], ignore_index= True)
print(data.shape)

(250000, 866)


In [40]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/pnc_250K_11_17.parquet", overwrite = True)